## Load data

In [35]:
import csv
import re
import numpy

class Repository(object):
    def __init__(self,fileName):
        self.fileName = fileName
    def  loadData(self,inputColumnFst,outputColumn,):
        data=[]
        columns=[]
        file = open(self.fileName)
        parsedData = csv.reader(file,delimiter=',')
       
        noLines = 0
        for line in parsedData:
            if noLines == 0:
                columns = line
            else:
                data.append(line)
            noLines += 1
                
        inputColumnIndexFst = columns.index(inputColumnFst)
        outputColumnIndex = columns.index(outputColumn)
    
        inputDataFst = []
        inputDataSnd = []
    
        outputData = []
        for j in range(len(data)):
            inputDataFst.append(data[j][inputColumnIndexFst])
            outputData.append(data[j][outputColumnIndex])
        
        file.close()
        
        return inputDataFst,outputData
    
    


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

filename = "textEmotions.csv"
repository = Repository(filename)
inputColumnFst = 'content'
vectorizer = TfidfVectorizer(stop_words='english')

outputColumn = 'sentiment'
inputDataResultFst,outputDataResult = repository.loadData(inputColumnFst, outputColumn)

print('inputs for column '+inputColumnFst +' is: \n'+str(inputDataResultFst)+'\n')
print('outputs for column '+outputColumn +' is: \n'+str(outputDataResult)+'\n')




inputs for column content is: 
['Layin n bed with a headache  ughhhh...waitin on your call...', 'Funeral ceremony...gloomy friday...', '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.', "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends", "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp\t he wants me 2! scandalous!", 'Hmmm. http://www.djhero.com/ is down', '@charviray Charlene my love. I miss you', "@kelcouch I'm sorry  at least it's Friday?", 'cant fall asleep', 'Choked on her retainers', 'Ugh! I have to beat this stupid song to get to the next  rude!', '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol', 'The storm is here and the electricity is gone', "So sleepy again and it's not even that late. I fail once again.", '@PerezHilton lady gaga tweeted about not being im

### separating training data from test data 20%-80%

In [37]:
import numpy as np 
import math

np.random.seed(5)

# generate the positions of the data that will be a part of the test data
noTestDataIndexes = math.floor( 0.2 * len(inputDataResultFst))
testDataIndexes = []
for index in range(0,noTestDataIndexes):
    testDataIndexes.append(np.random.randint(0,len(inputDataResultFst)))

inputTestFst = []
outputTest = []
inputTrainingFst = []
outputTraining = []
for i in range (0,len(inputDataResultFst)):
    if i in testDataIndexes :
        inputTestFst.append(inputDataResultFst[i])
        outputTest.append(outputDataResult[i])
    else:
        inputTrainingFst.append(inputDataResultFst[i])
        outputTraining.append(outputDataResult[i])
    
print(inputTrainingFst)
print(outputTraining)
print(inputTestFst)
print(outputTest)


def clean_text(review_text):
    import re
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()   
    return letters_only      
        
def apply_cleaning_function_to_series(X):
    print('Cleaning data')
    cleaned_X = []
    for element in X:
        cleaned_X.append(clean_text(element))
    print ('Finished')
    return cleaned_X

trainClean = apply_cleaning_function_to_series(inputTrainingFst)
testClean = apply_cleaning_function_to_series(inputTestFst)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 stop_words = None,
                                 ngram_range = (1,2),
                                 max_features = 200) 
trainData = vectorizer.fit_transform(trainClean)
testData = vectorizer.fit_transform(testClean)
trainData = trainData.toarray()
testData = testData.toarray()
tfidf = TfidfTransformer()
featuresTrain = tfidf.fit_transform(trainData).toarray()
featuresTest =  tfidf.fit_transform(testData).toarray()
vocab = vectorizer.get_feature_names()

['Funeral ceremony...gloomy friday...', '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.', "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends", "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp\t he wants me 2! scandalous!", '@charviray Charlene my love. I miss you', 'Choked on her retainers', 'Ugh! I have to beat this stupid song to get to the next  rude!', '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol', 'The storm is here and the electricity is gone', "So sleepy again and it's not even that late. I fail once again.", '@PerezHilton lady gaga tweeted about not being impressed by her video leaking just so you know', 'How are YOU convinced that I have always wanted you? What signals did I give off...damn I think I just lost another friend', "@raaaaaaek oh too bad! I h

    **Multivariate linear-regression using LogRegression (class)**

In [38]:
from math import exp
import random
class LogRegression(object):
    def __init__(self):
        self.w0 = 0.0
        self.w = []
        
    def sigmoid(self,x):
        return 1 / (1 + exp(-x))
    
    def calculateGuess(self,inputsRow):
        # getting the value of wo in the guess
        guess = self.w[0]
        for feature in range(len(inputsRow)):
            guess += self.w[feature+1] * inputsRow[feature]
        return guess
   
    def calculateError(self,guess,real):
        return guess - real
        
    def computeCoef0PerSample(self,error,learningRate):
        return self.w[0] - error * learningRate
    
    def computeCoefsPerSample(self,error,learningRate,sampleValue,noCoef):
        return self.w[noCoef] - error * learningRate * sampleValue
        
    # stochastic gradient descent
    def fit(self,inputs,outputs,learningRate = 0.001,noEpochs = 1000):
        noOfFeatures = len(inputs[0])+1
        for _ in range(noOfFeatures):
            self.w.append(0.0)
        # for _ in range(noOfFeatures):
            # self.w.append(random.random())
        for epoch in range(noEpochs):
            # ceofficients update with each sample
            for s in range(len(inputs)):
                guess = self.sigmoid(self.calculateGuess(inputs[s]))
                crtError = self.calculateError(guess,outputs[s])
                for noCoef in range(0,len(inputs[0])):
                    self.w[noCoef+1] = self.w[noCoef+1] - crtError * learningRate * inputs[s][noCoef]
                self.w[0] = self.w[0] - crtError * learningRate 
        self.w0 = self.w[0]
        self.w = self.w[1:]
    
    def eval(self,inputsRow):
         # getting the value of wo in the guess
        guess = self.w0
        for feature in range(len(inputsRow)):
            guess += self.w[feature] * inputsRow[feature]
        return guess
    
    def predict(self,inputs):
        outputsComputed = []
        for i in range(0,len(inputs)):
            outputsComputed.append(self.sigmoid(self.eval(inputs[i])))
        return outputsComputed

### leran model

In [39]:
def transformOneVsAll(oneClass):
    trainInputsAll = []
    trainOutputsAll = []
    testInputsAll = []
    testOutputsAll = []
    for dt in range(len(featuresTrain)):
        trainInputsAll.append(featuresTrain[dt])
        if outputTraining[dt] == oneClass:
            trainOutputsAll.append(1)
        else:
            trainOutputsAll.append(0)
            
    for dt in range(len(featuresTest)):
        testInputsAll.append(featuresTest[dt])
        if outputTest[dt] == oneClass:
            testOutputsAll.append(1)
        else:
            testOutputsAll.append(0)    
    return trainInputsAll,trainOutputsAll,testInputsAll,testOutputsAll

trainInputsAll1,trainOutputsAll1,testInputsAll1,testOutputsAll1 = transformOneVsAll('worry')
trainInputsAll2,trainOutputsAll2,testInputsAll2,testOutputsAll2 = transformOneVsAll('sadness')
trainInputsAll3,trainOutputsAll3,testInputsAll3,testOutputsAll3 = transformOneVsAll('neutral')

from sklearn import linear_model
classifierTool1 = linear_model.LogisticRegression()
classifierTool2 = linear_model.LogisticRegression()
classifierTool3 = linear_model.LogisticRegression()
classifierTool1.fit(trainInputsAll1, trainOutputsAll1)
classifierTool2.fit(trainInputsAll2, trainOutputsAll2)
classifierTool3.fit(trainInputsAll3, trainOutputsAll3)

w01Tool, w11Tool, w21Tool, w31Tool,w41Tool  = classifierTool1.intercept_[0],classifierTool1.coef_[0][0],classifierTool1.coef_[0][1],classifierTool1.coef_[0][2],classifierTool1.coef_[0][3]
w02Tool, w12Tool, w22Tool, w32Tool,w42Tool  = classifierTool2.intercept_[0],classifierTool2.coef_[0][0],classifierTool2.coef_[0][1],classifierTool2.coef_[0][2],classifierTool2.coef_[0][3]
w03Tool, w13Tool, w23Tool, w33Tool,w43Tool  = classifierTool3.intercept_[0],classifierTool3.coef_[0][0],classifierTool3.coef_[0][1],classifierTool3.coef_[0][2],classifierTool3.coef_[0][3]
print('classification model for class worry : y(feat1, feat2,feat3,feat4) = ', w01Tool, ' + ', w11Tool, ' * feat1 + ', w21Tool, ' * feat2 + ',w31Tool, ' * feat3 + ',w41Tool, ' * feat4')
print('classification model for class sadness : y(feat1, feat2,feat3,feat4) = ', w02Tool, ' + ', w12Tool, ' * feat1 + ', w22Tool, ' * feat2 + ',w32Tool, ' * feat3 + ',w42Tool, ' * feat4')
print('classification model for class neutral : y(feat1, feat2,feat3,feat4) = ', w03Tool, ' + ', w13Tool, ' * feat1 + ', w23Tool, ' * feat2 + ',w33Tool, ' * feat3 + ',w43Tool, ' * feat4')


classifier1 = LogRegression()
#classifier1.fit(trainInputsAll1,trainOutputsAll1,0.001,1100)

classifier2 = LogRegression()
#classifier2.fit(trainInputsAll2,trainOutputsAll2,0.001,1100)

classifier3 = LogRegression()
#classifier3.fit(trainInputsAll3,trainOutputsAll3,0.001,1100)

#w01, w11, w21, w31,w41  = classifier1.w0,classifier1.w[0],classifier1.w[1],classifier1.w[2],classifier1.w[3]
#w02, w12, w22, w32,w42  = classifier2.w0,classifier2.w[0],classifier2.w[1],classifier2.w[2],classifier2.w[3]
#w03, w13, w23, w33,w43  = classifier3.w0,classifier3.w[0],classifier3.w[1],classifier3.w[2],classifier3.w[3]
#print('Code:')
#print('classification model for class worry : y(feat1, feat2,feat3,feat4) = ', w01, ' + ', w11, ' * feat1 + ', w21, ' * feat2 + ',w31, ' * feat3 + ',w41, ' * feat4')
#print('classification model for class sadness : y(feat1, feat2,feat3,feat4) = ', w02, ' + ', w12, ' * feat1 + ', w22, ' * feat2 + ',w32 ,' * feat3 + ',w42, ' * feat4')
#print('classification model for class neutral : y(feat1, feat2,feat3,feat4) = ', w03, ' + ', w13, ' * feat1 + ', w23, ' * feat2 + ',w33, ' * feat3 + ',w43, ' * feat4')




classification model for class worry : y(feat1, feat2,feat3,feat4) =  -0.6071726539152065  +  0.7697575899119514  * feat1 +  -0.13074777626696174  * feat2 +  0.05706478393009234  * feat3 +  -0.0008612288151850366  * feat4
classification model for class sadness : y(feat1, feat2,feat3,feat4) =  -0.9218933363720749  +  -0.15474433856474937  * feat1 +  0.3514939624605694  * feat2 +  0.2865078020411799  * feat3 +  -0.2150058187014043  * feat4
classification model for class neutral : y(feat1, feat2,feat3,feat4) =  -0.6562340412768515  +  -0.6252320146591644  * feat1 +  -0.21662525079326453  * feat2 +  -0.33822586482988215  * feat3 +  0.21340071093651852  * feat4


## predict classes for test data

In [40]:
computedTestOutputs1 = classifierTool1.predict(testInputsAll1)
computedTestOutputs2 = classifierTool2.predict(testInputsAll2)
computedTestOutputs3 = classifierTool3.predict(testInputsAll3)


computedTestOutputs = []

for i in range(0,len(computedTestOutputs1)):
    max = computedTestOutputs1[i]
    cls = 'worry'
    if computedTestOutputs2[i] > max :
        max = computedTestOutputs2[i]
        cls = 'sadness'
    if computedTestOutputs3[i] > max :
        max = computedTestOutputs3[i]
        cls = 'neutral'
    computedTestOutputs.append(cls)
    




### calculate error



In [41]:
error = 0.0
for guess, real in zip(computedTestOutputs, outputTest):
    if guess != real:
        error += 1
error = error / len(outputTest)
print("classification error (manual) with code: ", error)


from sklearn.metrics import accuracy_score
error = 1 - accuracy_score(outputTest, computedTestOutputs)
print("classification error (tool) with code: ", error)

classification error (manual) with code:  0.7741935483870968
classification error (tool) with code:  0.7741935483870968
